In [1]:
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen

In [44]:
fp = '/Users/alexander.fioto/Models/Labeled-Seinfeld-Model/' 

In [48]:
ai = aitextgen(model= fp + "pytorch_model.bin",
               config = fp + 'config.json', 
               vocab_file=fp + 'aitextgen-vocab.json',
               merges_file=fp + 'aitextgen-merges.txt')

INFO:aitextgen:Loading GPT-2 model from provided /Users/alexander.fioto/Models/Labeled-Seinfeld-Model/pytorch_model.bin.
INFO:aitextgen:Using a custom tokenizer.


In [108]:
ai.generate(prompt='ALEX: tell me about yourself',
            temperature = .01)

ALEX: tell me about yourself.


JERRY: What?


GEORGE: You're going to get me a sample of Jerry


In [126]:
class SeinfeldChatbot():
    def __init__(self, name='USER: ', fp='/Users/alexander.fioto/Models/Labeled-Seinfeld-Model/', temperature = .4):
        self.user_name_title = name
        self.fp = fp
        self.temperature = temperature
        self.exit_commands = ['bye', 'exit', 'i have to go', 'later', 'gtg', 'stop']
        self.punctuation = ['.', '!', '?']
        self.model = aitextgen(model= fp + "pytorch_model.bin",
                               config = fp + 'config.json', 
                               vocab_file=fp + 'aitextgen-vocab.json',
                               merges_file=fp + 'aitextgen-merges.txt')
        print("Model Loaded!")
        
    
    def change_temp(self, temp):
        if temp < 0 or temp > 1:
            raise ValueError('Value must be greater than 0 and less than or equal to 1')
        else:
            self.temperature = temp
        
        
    def get_name(self):
        name = input("GEORGE: What\'s your name, pal?")
        self.name = name
        self.user_name_title = name.upper() + ': '
    
    def chat(self):
        self.get_name()
        print(f'JERRY: What do you want, {self.name}?')
        chat = True
        while chat:
            text_input = input()
            if text_input in self.exit_commands:
                chat = False
                print('KRAMER: Who turns down a junior mint?')
                
            else:
                if text_input[-1] not in self.punctuation:
                    text_input += '.'
                self.model.generate(prompt = f'{self.user_name_title}' + text_input,
                                    temperature = .4)

        
        
        

In [127]:
bot = SeinfeldChatbot()

INFO:aitextgen:Loading GPT-2 model from provided /Users/alexander.fioto/Models/Labeled-Seinfeld-Model/pytorch_model.bin.
INFO:aitextgen:Using a custom tokenizer.


Model Loaded!


In [128]:
bot.chat()

GEORGE: What's  your name, pal? Alex


JERRY: What do you want, Alex?


 I want to talk to you!


ALEX: I want to talk to you!


JERRY: But you said, it can be very serious.


ELAINE: Okay,


 This is silly.


...

ALEX: I think you should. Okay, bye.


JERRY: Why don't you mind your own business?


KRAMER: I


 How hot is it outside


ALEX: How hot is it outside.


JERRY: Oh, it's a better thing than that.


GEORGE:  Oh,


 stop


KRAMER: Well I'm out of here too!
